# 目的
bitFlyerが公開している値とCCCAGGの公開している値を比較する。

In [ ]:
import sys
import pyBitCoin.pybitcoin as pybct
import pickle
import time

import matplotlib.pyplot as plt
# from matplotlib.finance import candlestick_ohlc
import mpl_finance
%matplotlib inline

# CCCAGGとbitFlyerのOHLCは似通っているか？

## bitFlyer

In [ ]:
# brands_interest = ["BTC", "ETH", "ETC", "LTC", "BCH", "MONA", "LSK"]
# brands_into = ["JPY", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC"]

brands_interest = ["BTC", "ETH", "BCH"] # BTCFX, ETC, LTC, MONA, LSK はない 
brands_into = ["JPY", "BTC", "BTC"]

dfs = [None] * len(brands_interest)

is_datetimestr = False
datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2016, 1, 1, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 4, 30, 23, 0, 0).timestamp()
histoticks = "hour"
days = 30
limit = int(24*days-1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii*days*24*3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    for jj in range(len(brands_interest)):
        print("{} -> {}".format(brands_interest[jj], brands_into[jj]))
        params = {
            "fsym": brands_interest[jj],
            "tsym": brands_into[jj],
            "limit": str(limit),
            "e": "bitFlyer",
            "toTs":str(toTs)
        }
        result = pybct.core.rategetter.getRateViaCrypto(histoticks, params)
        if ii == 0:
            dfs[jj] = pybct.core.rategetter.toDataFrame(result, is_datetimestr)
        else:
            dfs[jj] = pd.concat([toDataFrame(result, is_datetimestr), dfs[jj]], ignore_index=True)
        time.sleep(10)
    break
#     if ii == 2:
#         break

In [ ]:
with open("../data/test.pickle", "wb") as ff:
    pickle.dump(dfs, ff)

In [ ]:
with open("../data/test.pickle", "rb") as ff:
    dfs_flyer = pickle.load(ff)

## CCCAGG

In [ ]:
# brands_interest = ["BTC", "ETH", "ETC", "LTC", "BCH", "MONA", "LSK"]
# brands_into = ["JPY", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC"]

brands_interest = ["BTC", "ETH", "BCH"] # BTCFX, ETC, LTC, MONA, LSK はない 
brands_into = ["JPY", "BTC", "BTC"]

dfs = [None] * len(brands_interest)

is_datetimestr = False
datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2016, 1, 1, 1, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 6, 1, 0, 0, 0).timestamp()
histoticks = "hour"
days = 30
limit = int(24*days-1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii*days*24*3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    for jj in range(len(brands_interest)):
        print("{} -> {}".format(brands_interest[jj], brands_into[jj]))
        params = {
            "fsym": brands_interest[jj],
            "tsym": brands_into[jj],
            "limit": str(limit),
#             "e": "bitFlyer",
            "toTs":str(toTs)
        }
        result = pybct.core.rategetter.getRateViaCrypto(histoticks, params)
        if ii == 0:
            dfs[jj] = pybct.core.rategetter.toDataFrame(result, is_datetimestr)
        else:
            dfs[jj] = pd.concat([toDataFrame(result, is_datetimestr), dfs[jj]], ignore_index=True)
        time.sleep(10)
    break
#     if ii == 2:
#         break

In [ ]:
with open("../data/test_cccagg.pickle", "wb") as ff:
    pickle.dump(dfs, ff)

In [ ]:
with open("../data/test_cccagg.pickle", "rb") as ff:
    dfs_cccagg = pickle.load(ff)

## Compare

In [ ]:

for jj in range(len(dfs_flyer)):
    timestamps = dfs_flyer[jj]["time"].values
    fig, axes = plt.subplots(figsize=(8, 4), ncols=2)
    _ = mpl_finance.candlestick_ohlc(axes[0], dfs_flyer[jj][["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)
    plt.grid()

    _ = mpl_finance.candlestick_ohlc(axes[1], dfs_cccagg[jj][["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)
    plt.grid()
    plt.tight_layout()

わりと傾向は似ている？

In [ ]:
index = 0
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

In [ ]:
index = 1
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

In [ ]:
index = 2
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

CCCAGGの値とbitFlyerの値はほぼ線形である様子。   
volumeは違うようである。   
傾向を確認するだけであれば、CCCAGGのトレンドを確認すればよいことになる。   
（ただし、これらは販売所の値でしかないことに注意すべき？）

CCCAGGの方が、公開されている銘柄のペアが多い。

# 分足ではどうか

## bitFlyer

In [ ]:
# brands_interest = ["BTC", "ETH", "ETC", "LTC", "BCH", "MONA", "LSK"]
# brands_into = ["JPY", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC"]

brands_interest = ["BTC", "ETH", "BCH"] # BTCFX, ETC, LTC, MONA, LSK はない 
brands_into = ["JPY", "BTC", "BTC"]

dfs = [None] * len(brands_interest)

is_datetimestr = False
datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2018, 5, 28, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 6, 2, 0, 0, 0).timestamp()
histoticks = "minute"
hours = 4 # ６回に分けてデータを取得する
limit = int(hours*60 - 1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii * hours * 3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    for jj in range(len(brands_interest)):
        print("{} -> {}".format(brands_interest[jj], brands_into[jj]))
        params = {
            "fsym": brands_interest[jj],
            "tsym": brands_into[jj],
            "limit": str(limit),
            "e": "bitFlyer",
            "toTs":str(toTs)
        }
        result = pybct.core.rategetter.getRateViaCrypto(histoticks, params)
        if ii == 0:
            dfs[jj] = pybct.core.rategetter.toDataFrame(result, is_datetimestr)
        else:
            dfs[jj] = pd.concat([toDataFrame(result, is_datetimestr), dfs[jj]], ignore_index=True)
        time.sleep(10)
    break
#     if ii == 2:
#         break

In [ ]:
with open("../data/test_minute.pickle", "wb") as ff:
    pickle.dump(dfs, ff)

In [ ]:
with open("../data/test_minute.pickle", "rb") as ff:
    dfs_flyer = pickle.load(ff)

## CCCAGG

In [ ]:
# brands_interest = ["BTC", "ETH", "ETC", "LTC", "BCH", "MONA", "LSK"]
# brands_into = ["JPY", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC"]

brands_interest = ["BTC", "ETH", "BCH"] # BTCFX, ETC, LTC, MONA, LSK はない 
brands_into = ["JPY", "BTC", "BTC"]

dfs = [None] * len(brands_interest)

is_datetimestr = False
datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2018, 5, 28, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 6, 2, 0, 0, 0).timestamp()
histoticks = "minute"
hours = 4 # ６回に分けてデータを取得する
limit = int(hours*60 - 1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii * hours * 3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    for jj in range(len(brands_interest)):
        print("{} -> {}".format(brands_interest[jj], brands_into[jj]))
        params = {
            "fsym": brands_interest[jj],
            "tsym": brands_into[jj],
            "limit": str(limit),
#             "e": "bitFlyer",
            "toTs":str(toTs)
        }
        result = pybct.core.rategetter.getRateViaCrypto(histoticks, params)
        if ii == 0:
            dfs[jj] = pybct.core.rategetter.toDataFrame(result, is_datetimestr)
        else:
            dfs[jj] = pd.concat([toDataFrame(result, is_datetimestr), dfs[jj]], ignore_index=True)
        time.sleep(10)
    break
#     if ii == 2:
#         break

In [ ]:
with open("../data/test_cccagg_minute.pickle", "wb") as ff:
    pickle.dump(dfs, ff)

In [ ]:
with open("../data/test_cccagg_minute.pickle", "rb") as ff:
    dfs_cccagg = pickle.load(ff)

## Compare

In [ ]:

for jj in range(len(dfs_flyer)):
    timestamps = dfs_flyer[jj]["time"].values
    fig, axes = plt.subplots(figsize=(8, 4), ncols=2)
    _ = mpl_finance.candlestick_ohlc(axes[0], dfs_flyer[jj][["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)
    plt.grid()

    _ = mpl_finance.candlestick_ohlc(axes[1], dfs_cccagg[jj][["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)
    plt.grid()
    plt.tight_layout()

In [ ]:
index = 0
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

In [ ]:
index = 1
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

In [ ]:
index = 2
plt.figure(figsize=(12,8))
plt.subplot(231)
plt.plot(dfs_flyer[index][["close"]].values, dfs_cccagg[index][["close"]].values, "o")
plt.grid()
plt.title("close")

plt.subplot(232)
plt.plot(dfs_flyer[index][["open"]].values, dfs_cccagg[index][["open"]].values, "o")
plt.grid()
plt.title("open")

plt.subplot(233)
plt.plot(dfs_flyer[index][["high"]].values, dfs_cccagg[index][["high"]].values, "o")
plt.grid()
plt.title("high")

plt.subplot(234)
plt.plot(dfs_flyer[index][["low"]].values, dfs_cccagg[index][["low"]].values, "o")
plt.grid()
plt.title("low")

plt.subplot(235)
plt.plot(dfs_flyer[index][["volumeto"]].values, dfs_cccagg[index][["volumeto"]].values, "o")
plt.grid()
plt.title("volumeto")

plt.tight_layout()

取引量がCCCAGGとbitFlyerでかなり異なる（bitFlyerのほうが少なすぎる）ため、分足の比較はあまりあてにならない様子。   